In [2]:
import pandas as pd
import glob
from functools import reduce

audl = pd.concat([pd.read_csv(dfteam, index_col=0) for dfteam in glob.glob('../output/*.csv')])

In [37]:
audl['Begin Area'].unique()
audl['Action'].unique()

array(['Catch', 'Throwaway', 'Goal', 'Pull', 'D', 'Drop',
       'EndOfFirstQuarter', 'Halftime', 'EndOfThirdQuarter', 'GameOver',
       'PullOb', 'Stall', 'Callahan', 'MiscPenalty', 'EndOfFourthQuarter',
       'EndOfOvertime', 'OpponentPull', 'OpponentCatch', 'OpponentPullOb'],
      dtype=object)

In [57]:
audl[audl.Action=='MiscPenalty'][['Passer','Receiver','Defender']].head()

,Passer,Receiver,Defender
1770,Adam Drews,NaN,NaN
3450,Adam Drews,NaN,NaN
3551,Davis Whitehead,NaN,NaN
4634,Ben Nelson,NaN,NaN
4739,NaN,NaN,NaN


In [87]:
audl[audl.Action.isin(['Pull','PullOb'])][['Passer','Defender','Receiver','End X']].head()
audl[audl.Action.isin(['Pull','PullOb'])&~pd.isnull(audl['End Y'])][['Begin Area',
       'Begin X', 'Begin Y', 'End Area', 'End X', 'End Y',
       'Distance Unit of Measure', 'Absolute Distance', 'Lateral Distance',
       'Toward Our Goal Distance']].head()

,Begin Area,Begin X,Begin Y,End Area,End X,End Y,Distance Unit of Measure,Absolute Distance,Lateral Distance,Toward Our Goal Distance
2044,Endzone B,0.203,0.586,Field,0.053,0.138,yds,83.3,23.9,79.9
2063,Endzone A,0.717,0.593,Endzone B,0.466,0.621,yds,95.0,1.5,95.0
2076,Endzone B,0.331,0.143,Field,0.111,0.482,yds,79.8,18.1,77.7
2088,Endzone A,0.803,0.706,Endzone B,0.293,0.558,yds,90.1,7.9,89.8
2126,Endzone A,0.752,0.711,Endzone B,0.386,0.338,yds,94.8,19.9,92.7


In [13]:
audl.columns

Index(['Date/Time', 'Opponent', 'Point Elapsed Seconds', 'Line',
       'Our Score - End of Point', 'Their Score - End of Point', 'Event Type',
       'Action', 'Passer', 'Receiver', 'Defender', 'Hang Time (secs)',
       'Player 0', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5',
       'Player 6', 'Player 7', 'Player 8', 'Player 9', 'Player 10',
       'Player 11', 'Player 12', 'Player 13', 'Player 14', 'Player 15',
       'Player 16', 'Player 17', 'Player 18', 'Player 19', 'Player 20',
       'Player 21', 'Player 22', 'Player 23', 'Player 24', 'Player 25',
       'Player 26', 'Player 27', 'Elapsed Time (secs)', 'Begin Area',
       'Begin X', 'Begin Y', 'End Area', 'End X', 'End Y',
       'Distance Unit of Measure', 'Absolute Distance', 'Lateral Distance',
       'Toward Our Goal Distance', 'Teamname', 'Tournament'],
      dtype='object')

In [92]:
def GetStat(df_slice, player_field,stat_name,aggfunc):
    stat = audl[df_slice].groupby(['Tournament',player_field,'Teamname']).apply(aggfunc)
    stat = stat.reset_index().rename(columns={'Tournament':'Year',player_field:'Name',0:stat_name})
    return stat

def GetCountStat(df_slice, player_field,stat_name):
    return GetStat(df_slice, player_field,stat_name,lambda x : len(x))

count_stat_list= [
                    # Catching
                    [(audl.Action=='Goal'), 'Receiver','Goals'],
                    [(audl.Action.isin(['Goal','Catch'])), 'Receiver','Catches'],
                    [(audl.Action=='Drop'), 'Receiver','Drops'],
                    # Throwing
                    [(audl.Action=='Goal'), 'Passer','Assists'],
                    [([True]*len(audl)), 'Passer','Throws'],
                    [(audl.Action.isin(['Goal','Catch'])), 'Passer','Completions'],
                    [(audl.Action.isin(['Throwaway','Callahan'])),'Passer','Throwaways'],
                    [(audl.Action=='Drop'), 'Passer','Throws Dropped'],
                    [(audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Absolute Distance'])), 'Passer','Throws Recorded with Yardage'],
                    [(audl.Action=='MiscPenalty'),'Passer','Fouls Against'],
                    [(audl.Action=='Stall'),'Passer','Stalls'],

                    # Defense
                    [(audl.Action.isin(['D','Callahan'])), 'Defender', 'Blocks'],
                    [(audl.Action=='Callahan'), 'Defender','Callahans'],
                    [audl.Action.isin(['Pull','PullOb']), 'Defender', 'Pulls'],
                    [(audl.Action=='PullOb'), 'Defender', 'Pulls (Out-of-bounds)'],
                    [audl.Action.isin(['Pull','PullOb'])&~pd.isnull(audl['Absolute Distance']), 'Defender','Pulls Recorded with Yardage'],
                    [(audl.Action=='Pull')&~pd.isnull(audl['Absolute Distance']), 'Defender','Pulls (Inbounds) Recorded with Yardage'],

]


other_stat_list = [
                    [audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Absolute Distance']),
                     'Passer',
                     'Throwing Yards',
                     lambda x : sum(x['Absolute Distance'])],
                    [audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Lateral Distance']),
                     'Passer',
                     'Lateral Throwing Yards',
                     lambda x : sum(x['Lateral Distance'])],
                    [audl.Action.isin(['Goal','Catch'])&~pd.isnull(audl['Toward Our Goal Distance']),
                     'Passer',
                     'Forward Throwing Yards',
                     lambda x : sum(x['Toward Our Goal Distance'])],

                    [audl.Action.isin(['Pull','PullOb'])&~pd.isnull(audl['Absolute Distance']),
                     'Defender',
                     'Pull Yards',
                     lambda x : sum(x['Absolute Distance'])],

                    [(audl.Action=='Pull')&~pd.isnull(audl['Absolute Distance']),
                     'Defender',
                     'Pull Yards (Inbounds)',
                     lambda x : sum(x['Absolute Distance'])]
    
                   ]

ind_stats = [GetCountStat(i,j,k) for i,j,k in count_stat_list] + \
            [GetStat(i,j,k,l) for i,j,k,l in other_stat_list]
    

base_stats = reduce(lambda  left,right: pd.merge(left,right,on=['Year','Teamname','Name'],
                                                 how='outer'), 
                                                 ind_stats).fillna(0)

In [98]:
def SafeDivide(x,y):
    if y!=0:
        return x/y
    else:
        return 0
    
secondary_stats = [ 
                    ['Completion Percentage', lambda x : 100*SafeDivide(x.Completions,x.Throws)],
                    ['Catches Per Goals', lambda x : SafeDivide(x.Catches,x.Goals)],
                    ['Throws Per Assist', lambda x : SafeDivide(x.Throws,x.Assists)],
                    ['Throwaway Percentage', lambda x : SafeDivide(x.Throwaways,x.Throws)],
                    ['Drop Percentage', lambda x : SafeDivide(x.Drops,(x.Drops + x.Catches))],  
                    ['Yards Per Throw', lambda x : SafeDivide(x['Throwing Yards'],x['Throws Recorded with Yardage'])],    
                    ['Yards Per Pull', lambda x : SafeDivide(x['Pull Yards'],x['Pulls Recorded with Yardage'])],  
                    ['Yards Per Pull (Inbounds)', lambda x : SafeDivide(x['Pull Yards (Inbounds)'],x['Pulls (Inbounds) Recorded with Yardage'])],    

                 ]

for sec_stat,aggfunc in secondary_stats:
    base_stats[sec_stat] = base_stats.apply(aggfunc,axis=1)

In [99]:
base_stats.sort_values('Throws Recorded with Yardage',ascending=False)
base_stats.sort_values('Fouls Against',ascending=False)
base_stats.sort_values('Throws Dropped',ascending=False)
base_stats.sort_values('Pull Yards',ascending=False)
base_stats.sort_values('Yards Per Throw',ascending=False)

# base_stats.groupby('Name').apply(sum).sort_values('Callahans',ascending=False)

,Year,Name,Teamname,Goals,Catches,Drops,Assists,Throws,Completions,Throwaways,...,Pull Yards,Pull Yards (Inbounds),Completion Percentage,Catches Per Goals,Throws Per Assist,Throwaway Percentage,Drop Percentage,Yards Per Throw,Yards Per Pull,Yards Per Pull (Inbounds)
3231,AUDL 2018,Zakarie Massi,Montreal Royal,1.0,3.0,1.0,2.0,2.0,2.0,0.0,...,0.0,0.0,100.000000,3.000000,1.000000,0.000000,0.250000,61.050000,0.000000,0.000000
2029,AUDL 2017,Dave Wiseman,Madison Radicals,18.0,219.0,6.0,21.0,203.0,175.0,26.0,...,111.6,111.6,86.206897,12.166667,9.666667,0.128079,0.026667,25.684615,55.800000,55.800000
3169,AUDL 2018,Ted Barnett,Los Angeles Aviators,1.0,22.0,2.0,3.0,39.0,32.0,5.0,...,2366.6,2226.2,82.051282,22.000000,13.000000,0.128205,0.083333,23.500000,81.606897,82.451852
1991,AUDL 2017,Christian Foster,Montreal Royal,5.0,56.0,0.0,9.0,58.0,51.0,6.0,...,1766.7,1593.3,87.931034,11.200000,6.444444,0.103448,0.000000,22.819231,88.335000,88.516667
2441,AUDL 2017,Ryan Tucker,Madison Radicals,22.0,51.0,1.0,4.0,29.0,28.0,0.0,...,0.0,0.0,96.551724,2.318182,7.250000,0.000000,0.019231,22.414286,0.000000,0.000000
796,AUDL 2015,Kevin Smith,San Jose Spiders,4.0,217.0,0.0,35.0,330.0,299.0,27.0,...,1608.4,1608.4,90.606061,54.250000,9.428571,0.081818,0.000000,22.368478,76.590476,76.590476
2297,AUDL 2017,Marquis Mason,Madison Radicals,5.0,11.0,1.0,0.0,7.0,7.0,0.0,...,0.0,0.0,100.000000,2.200000,0.000000,0.000000,0.083333,22.200000,0.000000,0.000000
2286,AUDL 2017,Mark Elbogen,Los Angeles Aviators,22.0,106.0,4.0,13.0,85.0,75.0,7.0,...,0.0,0.0,88.235294,4.818182,6.538462,0.082353,0.036364,22.195082,0.000000,0.000000
2243,AUDL 2017,Kevin Groulx,Montreal Royal,20.0,161.0,4.0,18.0,150.0,116.0,28.0,...,4451.8,4118.7,77.333333,8.050000,8.333333,0.186667,0.024242,22.150000,80.941818,80.758824
2608,AUDL 2018,Antoine Genest,Montreal Royal,9.0,141.0,2.0,29.0,219.0,183.0,31.0,...,1778.4,1609.6,83.561644,15.666667,7.551724,0.141553,0.013986,22.044681,71.136000,69.982609
